<a href="https://colab.research.google.com/github/itdusty/cells_generator/blob/main/cells_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
from PIL import Image, ImageDraw, ImageFilter
import os
from random import randint
import pandas as pd
import numpy as np

In [ ]:
if not os.path.isdir("dataset"): #Создаем основной репозиторий
    os.mkdir("dataset")
os.chdir("dataset") # Указываем используемый репозиторий

# Общие настройки
pack = 5 # Количество папок с изображениями
n_step = 5 # Количество изображений в папке
pix_x = 32 # Размер изображения в пикселях
pix_y = 32
debug = False # При значении True размер и количество клеток устанавливаются вручную
is_blurry = False # Включение размытия
is_noisy = False # Включение шума
delta = 1 # Шаг перемещения клеток

# Параметры, действующие при debug = True
sizeR = 1 # Размер точек
sizeG = 3
red_min = 8 # Минимальное количество эритроцитов на изображении
red_max = 20 # Максимальное количество эритроцитов на изображении
gray_min = 5 # Минимальное количество лимфоцитов на изображении
gray_max = 15 # Максимальное количество лимфоцитов на изображении

# Параметры, действующие при debug = False
locR = 1 # Среднее ("центр") нормального распределения для размеров эритроцитов (R) и лейкоцитов (G)
locG = 1.5
scaleR = 0.2 # Стандартное отклонение (разброс или "ширина") нормального распределения для размеров эритроцитов (R) и лейкоцитов (G)
scaleG = 0.5
loc_countR = 1 # Среднее ("центр") нормального распределения для количества эритроцитов (R) и лейкоцитов (G). Произведение со следующими коэффициентами
# определяет количество клеток
loc_countG = 1
scale_countR = 0.5 # Стандартное отклонение (разброс или "ширина") нормального распределения для количества эритроцитов (R) и лейкоцитов (G)
scale_countG = 0.5
cells_multiplierR = 10 # Коэффициенты, изменяющие разряд случайного числа. Полученное случайное число будет умножено на соответствующий коэффициент,
# тем самым увеличивая количество эритроцитов (R) и лейкоцитов (G)
cells_multiplierG = 10

# Настройка изображения
color_f = 'white' # Цвет фона изображения
format_image = '.png' # Формат сохраняемого изображения
name_image = 'broun' # Имя изображения
name_files = "traffic" #Имя папки
color_ery = 'red' # Цвет эритроцитов
color_limf = 'grey' # Цвет лимфоцитов
blur_radius = 0.5
noise_cutoff = 1 # Если случайное значение с нормальным распределением будет выше данного значения - на изображении появится "плохой" пиксель,
# если ниже - без изменений. Для noise_cutoff = 1 вероятность пояявления шума составляет 0.1587
noise_prob = 0.5 # Вероятность добавления шума: необходима для уменьшения вероятности появления шума.
# При значении 1 появление шума зависит только от noise_cutoff
noise_color = "random" # Цвет шума: grey или random

ran_coord_rx = [] #Создание используемых массивов
ran_coord_ry = []
ran_coord_gx = []
ran_coord_gy = []
Wxr = []
Wyr = []
Wxg = []
Wyg = []
files = []
erythrocytes = []
lymphocytes = []
noise = int(is_noisy)
blur = int(is_blurry)

for h in range(1, pack+1): # Цикл для создания папок в основном репозитории
    hh = str(h)
    if not os.path.isdir(name_files+hh): #Создание папки
        os.mkdir(name_files+hh)
    os.chdir(name_files+hh) # Название папки
    rep = name_files+hh
    print(os.getcwd()) # Проверка рабочей папки
    if debug:
      ran_red = randint(red_min, red_max) #Количество точек в одной папке
      ran_gray = randint(gray_min, gray_max)
    else:
      sizeR = round(abs(np.random.normal(loc=locR, scale=scaleR)))
      sizeG = round(abs(np.random.normal(loc=locG, scale=scaleG)))
      ran_red = round(abs(np.random.normal(loc=loc_countR, scale=scale_countR))*cells_multiplierR)
      ran_gray = round(abs(np.random.normal(loc=loc_countG, scale=scale_countG))*cells_multiplierG)
    files.append(rep)
    erythrocytes.append(ran_red)
    lymphocytes.append(ran_gray)

    for i in range(0,n_step):
        img = Image.new('RGBA', (pix_x, pix_y), color_f) # Создание изображения
        idraw = ImageDraw.Draw(img) # Создание объекта изображения
        Wxr.clear() # Обнуление списков
        Wyr.clear()
        Wxg.clear()
        Wyg.clear()

        if i == 0: # Генерация начального положения эритроцитов на изображении
            for m in range(0, ran_red):
                ran_coord_rx.append(float(random.uniform(0, pix_x)))
                ran_coord_ry.append(float(random.uniform(0, pix_y)))
        for a in range(0, ran_red): # Цикл для отображения эритроцитов
            ran_coord_rx.append(float(random.uniform((ran_coord_rx[0] - delta), (ran_coord_rx[0] + delta)))) # Определение следующей координаты
            ran_coord_ry.append(float(random.uniform((ran_coord_ry[0] - delta), (ran_coord_ry[0] + delta))))
            ran_coord_rx.remove(ran_coord_rx[0])
            ran_coord_ry.remove(ran_coord_ry[0])
            yi = np.random.choice([1, -1])
            Wxr.append(np.ones(n_step) * ran_coord_rx[0])
            Wyr.append(np.ones(n_step) * ran_coord_ry[0])
            Wxr[0] = Wxr[a-1] + (yi / np.sqrt(n_step)) # Уравнение броуновского движения
            Wyr[0] = Wyr[a - 1] + (yi / np.sqrt(n_step))
            wxr1 = Wxr[0]
            wxr = wxr1[0]
            wyr1 = Wyr[0]
            wyr = wyr1[0]
            xr1 = wxr - sizeR
            yr1 = wyr - sizeR
            xr2 = wxr + sizeR
            yr2 = wyr + sizeR
            idraw.ellipse((xr1, yr1, xr2, yr2), color_ery) # Отображение частицы
            istr = str(i)
            img.save(name_image + istr + format_image) # Сохраннение изображения

        if i == 0: # Генерация начального положения лейкоцитов на изображении
            for z in range(0, ran_gray):
                ran_coord_gx.append(float(randint(0, pix_x)))
                ran_coord_gy.append(float(randint(0, pix_y)))
        for p in range(0, ran_gray): # Цикл для отображения лейкоцитов
            ran_coord_gx.append(float(random.uniform((ran_coord_gx[0] - delta), (ran_coord_gx[0] + delta)))) # Определение следующей координаты
            ran_coord_gy.append(float(random.uniform((ran_coord_gy[0] - delta), (ran_coord_gy[0] + delta))))
            ran_coord_gx.remove(ran_coord_gx[0])
            ran_coord_gy.remove(ran_coord_gy[0])
            yi = np.random.choice([1, -1])
            Wxg.append(np.ones(n_step) * ran_coord_gx[p])
            Wyg.append(np.ones(n_step) * ran_coord_gy[p])
            Wxg[0] = Wxg[p - 1] + (yi / np.sqrt(n_step)) # Уравнение броуновского движения
            Wyg[0] = Wyg[p - 1] + (yi / np.sqrt(n_step))
            wxg1 = Wxg[0]
            wxg = wxg1[0]
            wyg1 = Wyg[0]
            wyg = wyg1[0]
            xg1 = wxg - sizeG
            yg1 = wyg - sizeG
            xg2 = wxg + sizeG
            yg2 = wyg + sizeG
            idraw.ellipse((xg1, yg1, xg2, yg2), color_limf) # Отображение частицы
            istr = str(i)
            img.save(name_image + istr + format_image) # Сохраннение изображения

        if is_blurry: # При включении размытия применяется Box Blur с радиусом blur_radius
            img = img.filter(ImageFilter.BoxBlur(blur_radius))
            idraw = ImageDraw.Draw(img)
            istr = str(i)
            img.save(name_image + istr + format_image)

        if is_noisy: # При включении шумов
            output = np.random.normal(size=(pix_x, pix_y))
            output[output < noise_cutoff] = 0 # Данные строки создают матрицу размером pix_x и pix_y, состоящую из 0 и 1, где 0 - "нормальный" пиксель, 1 - вероятно "плохой" пиксель
            output[output > noise_cutoff] = 1
            for x in range(pix_x):
              for y in range(pix_y):
                if output[x][y] == 1:
                  if random.uniform(0, 1) < noise_prob: # Если случайное число в диапазоне от 0 до 1 окажется меньше, чем noise_prob, то текущий пиксель является "плохим" (появляется шум)
                    if noise_color == "grey":
                      color_part = random.randint(0, 255)
                      color = (color_part, color_part, color_part) # Цифровой шум серого цвета разной интенсивности
                    elif noise_color == "random":
                      color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) # Цифровой шум случайного цвета
                    idraw.rectangle(((x, y), (x, y)), color) # Отображение шума
            istr = str(i)
            img.save(name_image + istr + format_image)

        print(name_image + istr + format_image)
    os.chdir("..") # Возвращение к исходному репозиторию
os.chdir("..")
frame = pd.DataFrame( {'files' : files, 'erythrocytes' : erythrocytes, 'lymphocytes' : lymphocytes, 'blur' : blur, 'noise' : noise})
print(frame)
frame.to_csv("./DataFrame.csv")

/content/dataset/traffic1
ran_coord_rx [140.9356669992233, 241.95683124537004, 131.9974155225882, 34.929812856676236, 198.31333432871807, 13.08122573266804, 62.64624542773151, 42.93503812453009, 93.3064173657288, 120.29302876271892, 92.07699754373613, 225.40366078137606, 144.91723263567388, 72.89585687290837, 241.11392502498032, 56.4106956464884, 146.76388601757006, 246.33370768926105, 161.29557422623589, 0.35853294800738955, 107.56768438166553, 193.99985640017832, 95.12831566451946, 54.213445293267135, 74.39390822412567, 182.91072917688777, 181.79523875854892, 153.27090323721492, 107.32929780621278, 187.65729773108615, 235.4743446883889, 110.84567277053853, 96.91155832961846, 236.05751487955396, 175.795217009206, 232.69614559935255, 101.88089313984808, 237.79352110628386, 169.21707094801602, 49.51456067877458, 145.850110823293, 185.97274654432476, 43.92876060931353, 55.533241724277396, 109.366469555042, 130.8317686390499, 123.06458817067076, 71.34909161484445, 5.833326059925481, 159.5